### Modelo de aprendizaje automático

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. 

In [2]:
# Importando librerias

import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

In [11]:
#Traigo la tabla para realizar el modelo
modelo_item= pd.read_csv('data/modelo_item.csv', encoding='utf8')
modelo_item.head()

,item_id,app_name,genres
0,719000,Strata Spaces VR – Professional Edition Upgrade,Design &amp; Illustration
1,719000,Strata Spaces VR – Professional Edition Upgrade,Animation &amp; Modeling
2,292180,X-Plane 10 Global - 64 Bit,Simulation
3,358040,HTC Vive,Sin Datos
4,629796,EVE Online: 15400 PLEX,RPG


Para obtener el modelo se deben convertir en dummies todos los generos disponibles

In [12]:
modelo_item_2= pd.get_dummies(modelo_item, columns=["genres"], prefix="")

In [16]:
modelo_item_2.head(3)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,719000,Strata Spaces VR – Professional Edition Upgrade,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,719000,Strata Spaces VR – Professional Edition Upgrade,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,292180,X-Plane 10 Global - 64 Bit,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
modelo_item_3= modelo_item_2.groupby(["item_id","app_name"]).sum().reset_index()
modelo_item_3.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
modelo_item_3.shape

(32132, 25)

In [25]:
modelo_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74833 entries, 0 to 74832
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   74833 non-null  int64 
 1   app_name  74833 non-null  object
 2   genres    74833 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


Aplicar modelo coseno

In [28]:
def recomendacion_juego(item_id):
    
    item_id = int(item_id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item_3[modelo_item_3['item_id'] == item_id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcular la matriz de similitud coseno
    similitudes = cosine_similarity(modelo_item_3.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item_3[modelo_item_3['item_id'] == item_id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item_3.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [30]:
recomendacion_juego(719000)

30198    Strata Spaces VR – Professional Edition Upgrade
26032                                Strata Design 3D SE
31261                            Substance Designer 2018
11665                                 Grand Designer Pro
16222                               Qubicle Voxel Editor
Name: app_name, dtype: object

In [32]:
modelo_final_csv = "modelo_final.csv"
modelo_item_3.to_csv(modelo_final_csv , index=False, encoding="utf-8")

In [33]:
funcion_6= pd.read_csv("modelo_final.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_6.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_6)
pq.write_table(table,output_file)